This is created by wakuphas on 2019.9.17.

For running YOLO training on GPU with using Google Colab.


# **1. Google Colab**

**あまりないが環境構築等で元に戻したくなったら、以下コマンドで初期化できる。**

In [0]:
!kill -9 -1

**12時間ルールは、下記shコマンドをセルから実行すれば、起動からの時間がわかる。**

12時間すぎると自動でGPUは切断される。数時間経つとばまた使用可能。ただし、アップロードしたファイルや出力されたファイルは全て削除されてしまうので、逐一重みファイルなどの出力ファイルはローカルに保存しておくことをおすすめする。

In [6]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'

0.456995days (39484.41sec)


#** 2. 環境構築**

## **CUDAをインストールする**

In [0]:
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

In [0]:
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

In [0]:
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub

In [0]:
!apt update

In [0]:
!apt install cuda-8-0 -y

In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

## **gcc-4.8を導入する**
gcc version 4.9以降は対応していないらしい

現在のバージョンを確認

In [0]:
!gcc -dumpversion

**gcc-4.8をインストール**

In [0]:
!apt-get install gcc-4.8 g++-4.8

In [0]:
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 40 --slave /usr/bin/g++ g++ /usr/bin/g++-4.8

!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-7 50 --slave /usr/bin/g++ g++ /usr/bin/g++-7

## **nvccをインストールする（nvidia-cuda-toolkit）**

In [0]:
!apt install nvidia-cuda-toolkit

# **3. 実行手順**

## ** Google Colabへdarknetをアップロード**


file_id='XXXXXXXXXXXXXXX'のXの部分にGoogle Driveにアップロードしたzipファイルの共有リンクアドレスを貼る。

アップしたファイルを右クリックし、「共有可能なリンクの取得」を選択すれば出てくる。

In [0]:
!pip install PyDrive googledrivedownloader

In [0]:
from google_drive_downloader import GoogleDriveDownloader

GoogleDriveDownloader.download_file_from_google_drive(file_id='XXXXXXXXXXXXXXXXXXXXX',
                                                      dest_path="./tmp.zip",
                                                      unzip=True)
!rm tmp.zip        
!ls

## **makeを通す**
**Makefileでは GPU=1 を確認**

In [0]:
# opencv = 1 にすると現状動かないが問題ない。

In [0]:
!cat darknet/Makefile

**darknet内でmakeする**

In [0]:
%%bash

cd darknet
make

## **darknetを用いて学習させる**
初期値：

yolo v2 のときは darknet19_448.conv.23

yolo v3 のときは darknet53.conv.74



一度中断しても出力された重みファイル.weightsを初期値として読み込めばそこから再開可能。


In [0]:
%%bash
cd darknet

# v2
./darknet detector train cfg/obj.data cfg/yolo-obj.cfg darknet19_448.conv.23 > train_log.txt
# 出力した重み yolo-obj_XXX.weights を読み込む場合
#./darknet detector train cfg/obj.data cfg/yolo-obj.cfg  backup/yolo-obj_XXX.weights > train_log.txt


# v3
#./airport detector train cfg/obj.data cfg/yolo-obj.cfg darknet53.conv.74


train_log.txtにログを出力。数行ごとに出てくる以下に着目。

--

**937**: 34.240753, **37.657242 avg**, 0.000771 rate, 5.082382 seconds, 59968 images

--


最初の数字が**反復回数**。通常2000回は必要。
avgの手前の数字が**小くなればなるほど精度が高い**。
0.1以下くらいが良い？

avgの数字が減らなくなってきたら学習を止める。

## **学習したモデルでテスト！**

現状なぜかCore dumpするのでスキップ。
tensorflowをMac上で動かして実行することにする。

In [0]:
%%bash
cd darknet

# v2
./darknet detector test cfg/yolo-obj.cfg backup/yolo-obj_900.weights ../test_airport.png
# v3
#./darknet detector test cfg/yolov3-airport.cfg bin/yolov3.weights test.png